In [2]:
import pandas as pd
data=pd.read_csv("/content/deliveries.csv")
data.head(10)

,match_id,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,batsman_runs,extra_runs,total_runs,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder
0,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.0,1.0,SC Ganguly,P Kumar,BB McCullum,0.0,1.0,1.0,legbyes,0.0,NaN,NaN,NaN
1,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.0,2.0,BB McCullum,P Kumar,SC Ganguly,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN
2,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.0,3.0,BB McCullum,P Kumar,SC Ganguly,0.0,1.0,1.0,wides,0.0,NaN,NaN,NaN
3,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.0,4.0,BB McCullum,P Kumar,SC Ganguly,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN
4,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.0,5.0,BB McCullum,P Kumar,SC Ganguly,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN
5,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.0,6.0,BB McCullum,P Kumar,SC Ganguly,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN
6,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0.0,7.0,BB McCullum,P Kumar,SC Ganguly,0.0,1.0,1.0,legbyes,0.0,NaN,NaN,NaN
7,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,1.0,1.0,BB McCullum,Z Khan,SC Ganguly,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN
8,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,1.0,2.0,BB McCullum,Z Khan,SC Ganguly,4.0,0.0,4.0,NaN,0.0,NaN,NaN,NaN
9,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,1.0,3.0,BB McCullum,Z Khan,SC Ganguly,4.0,0.0,4.0,NaN,0.0,NaN,NaN,NaN


In [6]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error

sixes_data = data[data['total_runs'] == 6]
sixes_per_match = sixes_data.groupby(['batter', 'match_id']).size().reset_index(name='sixes_count')

batsman = "BB McCullum"
batsman_data = sixes_per_match[sixes_per_match['batter'] == batsman]


time_series = batsman_data[['match_id', 'sixes_count']].set_index('match_id')
time_series.index = pd.date_range(start='2020-01-01', periods=len(time_series), freq='D')


train_size = int(len(time_series) * 0.8)
train_data = time_series[:train_size]
test_data = time_series[train_size:]

# Fit SARIMA model
sarima_model = SARIMAX(train_data, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12)).fit()
sarima_forecast = sarima_model.forecast(steps=len(test_data))

# Fit ARIMA model
arima_model = ARIMA(train_data, order=(1, 1, 1)).fit()
arima_forecast = arima_model.forecast(steps=len(test_data))

# Evaluate SARIMA model
rmse_sarima = np.sqrt(mean_squared_error(test_data, sarima_forecast))
mae_sarima = mean_absolute_error(test_data, sarima_forecast)

# Evaluate ARIMA model
rmse_arima = np.sqrt(mean_squared_error(test_data, arima_forecast))
mae_arima = mean_absolute_error(test_data, arima_forecast)

# Print evaluation metrics
print("SARIMA RMSE:", rmse_sarima)
print("SARIMA MAE:", mae_sarima)
print("ARIMA RMSE:", rmse_arima)
print("ARIMA MAE:", mae_arima)

# Determine better model
if rmse_sarima < rmse_arima and mae_sarima < mae_arima:
    print("SARIMA model performs better.")
elif rmse_arima < rmse_sarima and mae_arima < mae_sarima:
    print("ARIMA model performs better.")
else:
    print("The models have comparable performance; consider other metrics.")


SARIMA RMSE: 1.2752382572472645
SARIMA MAE: 0.9326959138947565
ARIMA RMSE: 1.344452045391098
ARIMA MAE: 1.089495172841725
SARIMA model performs better.


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
